### load model MentalAId

In [1]:
import numpy as np
import tensorflow as tf
import os
import pandas as pd
from sklearn import metrics
from sklearn.metrics import roc_auc_score

2024-11-06 15:43:13.844767: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

tf.config.threading.set_inter_op_parallelism_threads(1)
tf.config.threading.set_intra_op_parallelism_threads(1)
tf.config.set_soft_device_placement = False
tf.config.experimental.set_memory_growth = True

gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])

2024-11-06 15:43:15.437603: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2024-11-06 15:43:15.451669: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:25:00.0 name: Tesla V100-PCIE-32GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2024-11-06 15:43:15.451706: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2024-11-06 15:43:15.454014: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2024-11-06 15:43:15.454055: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2024-11-06 15:43:15.456387: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.

In [3]:
os.chdir('/data/home/lmx/psy_test/Github_MentalAId')

In [4]:
MentalAId = tf.keras.models.load_model('model/')

2024-11-06 15:43:15.579353: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-06 15:43:15.586337: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2700000000 Hz
2024-11-06 15:43:15.586589: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5f6c7e0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-11-06 15:43:15.586619: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2024-11-06 15:43:15.743595: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x6998a40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-11-06 

In [5]:
x_data = np.load('x_data.npy',allow_pickle=True)
y_data = np.load('y_data.npy',allow_pickle=True)
print(x_data.shape)
x_data = np.expand_dims(x_data, -1)
x_data = x_data.astype('float32') 

(60, 7, 7)


In [6]:
data = pd.read_csv('example_data.csv', index_col='Unnamed: 0')

sex_data = data['Sex']
sex_data.value_counts()
sex_data = np.array(sex_data).astype(np.float32)

age_data = data['Age']

age_data = np.array(age_data).astype(np.float32)

In [7]:
pred = MentalAId.predict([x_data, age_data, sex_data])

2024-11-06 15:43:19.317052: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2024-11-06 15:43:19.616704: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2024-11-06 15:43:19.856802: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


In [8]:
auc = roc_auc_score(y_data, pred)
print(f'> AUC: %.3f' % (auc))

> AUC: 0.989


In [9]:
pred_bin = (pred[:,1] >= 0.5).astype(int)
y_data_bin = np.argmax(y_data,axis=1)

acc = metrics.accuracy_score(y_data_bin, pred_bin)
prec = metrics.precision_score(y_data_bin, pred_bin)
rec = metrics.recall_score(y_data_bin, pred_bin)

cm = metrics.confusion_matrix(y_data_bin,pred_bin) # confusion matrix
spec = cm[0,0]/(cm[0,0]+cm[0,1])
mcc = metrics.matthews_corrcoef(y_data_bin, pred_bin)

print('MentalAId prediction performance:')
print(f'> Accuracy: %.3f ' % acc)
print(f'> Precision: %.3f ' % prec)
print(f'> Recall: %.3f ' % rec)
print(f'> Specificity: %.3f ' % spec)
print(f'> MCC: %.3f ' % mcc)

MentalAId prediction performance:
> Accuracy: 0.967 
> Precision: 1.000 
> Recall: 0.920 
> Specificity: 1.000 
> MCC: 0.933 
